# **Róbert Šafár** & **Matúš Totcimak**
## 1. fáza: *Prieskumná analýza - EDA*
#### Dataset 82
#### Podiel práce 50:50
#
#

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import re

## Načítanie CSV súborov

In [2]:
con = pd.read_csv("dataset82/connections.csv", sep='\t')
proc = pd.read_csv("dataset82/processes.csv", sep='\t')
dev = pd.read_csv("dataset82/devices.csv", sep='\t')
prof = pd.read_csv("dataset82/profiles.csv", sep='\t')

#
# **1.1 Základný opis dát spolu s ich charakteristikami**

#
## *A) Analýza štruktúr dát ako súbory, záznamy*

### Prvý pohľad na dáta.

#### Connections

In [ ]:
con.head()

#### Processes

In [ ]:
proc.head()

#### Devices

In [ ]:
dev.head()

#### Profiles

In [ ]:
prof.head()

### Zobrazenie základných informácií o jednotlivých DF.

#### Connections

In [ ]:
con.info()

#### Processes

In [ ]:
proc.info()

#### Devices

In [ ]:
dev.info()

#### Profiles

In [ ]:
prof.info()

### Zistíme počet zariadení, ktoré majú pridelené záznamy.

In [ ]:
print(f"Connections: {con["imei"].nunique()}\nProcesses: {proc["imei"].nunique()}\nDevices: {dev["imei"].nunique()}\nProfiles: {prof["imei"].nunique()}")

### Zistili sme že vo všetkých DF okrem `Profiles` máme záznamy o 500 zariadeniach.
#### Nevieme však povedať, či sa jedná o tie isté zariadenia, a ak sa aj jedná o tie isté zariadenia, nevieme pre ktoré nám chýba záznam v `Profiles`.

### Uistíme sa, či sa jedná o tie isté zariadenia.
### Vytvoríme DFs obsahujúce iba unikátne `imei` zo všetkých 4 DF.
### Následne ich spojíme práve na základe `imei` a overíme, či sa jedná o tie isté zariadenia.

In [ ]:
imei1 = pd.DataFrame({"imei": con["imei"].unique()})
imei2 = pd.DataFrame({"imei": proc["imei"].unique()})
imei3 = pd.DataFrame({"imei": dev["imei"].unique()})
imei4 = pd.DataFrame({"imei": prof["imei"].unique()})
m1 = pd.merge(imei1, imei2, on="imei")
m2 = pd.merge(m1, imei3, on="imei")
m3 = pd.merge(m1, imei4, on="imei")
print("Connections & Processes =", m1.shape[0], "\n+ Devices =", m2.shape[0], "\n+ Profiles =", m3.shape[0])

### Vidíme, že sa jedná o tie isté `imei`, iba v Profiles jedno chýba. Zistíme, ktoré konkrétne to je.

### Porovnáme unikátne `imei` napr. z `Connections` a `Profiles`, aby sme našli `imei`, ktoré sa nespojilo vyššie.
### Aby sme sa uistile že rovnaké `imei` sa nachádzajú na rovnakých pozíciách, naskôr si oba DF zoradíme.

In [ ]:
m3.sort_values(by="imei", inplace=True)
m3.reset_index(inplace=True)
imei1.sort_values(by="imei", inplace=True)
imei1.reset_index(inplace=True)
for i in range(0, 498):
    im1 = imei1.iloc[i]
    im2 = m3.iloc[i]
    if imei1.iloc[i, 1] != m3.iloc[i, 1]:
        print("Chýbajúce imei z Profiles: ", imei1.iloc[i, 1])
        break

### Základné deskriptívne hodnoty jednotlivých DF.

#### Connections

In [ ]:
con.describe()

#### Processes

In [ ]:
proc.describe()

#### Devices

In [ ]:
dev.describe()

#### Profiles

In [ ]:
prof.describe()

#
## *B) Analýza jednotlivých atribútov*

### Atribúty sme vyberali z DF: `Connections` a `Processes`.
### Pred samotnou analýzou vybraných atribútov sme overili možné duplikáty, ktoré by mohli ovplyvniť ďalšiu analýzu.

### Počet duplikátov v Connections.

In [ ]:
con.duplicated().sum()

### Počet duplikátov v Processes.

In [ ]:
proc.duplicated().sum()

### V oboch DF sa našli duplikátne záznamy -> odstránime ich.

#### Pomocná funkcia pre budúce porovnanie vymazaných záznamov.

In [20]:
def count_of_rows():
    return [con.shape[0], proc.shape[0], dev.shape[0], prof.shape[0]]

#### Zaznamenanie počtu záznamov pre budúce porovnanie.

In [21]:
before = count_of_rows()

### Odstránenie duplikátnych záznamov pre oba DF.

In [22]:
con = con.loc[~con.duplicated()].reset_index(drop=True).copy()
proc = proc.loc[~proc.duplicated()].reset_index(drop=True).copy()

### Výber konkrétnych atribútov na ďalšiu analýzu.

In [23]:
attributes = pd.DataFrame({
    'c.dogalize': con['c.dogalize'],
    'c.katana': con['c.katana'],
    'c.android.gm': con['c.android.gm'],
    'p.android.packageinstaller': proc['p.android.packageinstaller'],
    'p.android.externalstorage': proc['p.android.externalstorage'],
    'p.system': proc['p.system'],
    'p.android.chrome': proc['p.android.chrome'],
    'p.android.settings': proc['p.android.settings'],
    'p.android.documentsui': proc['p.android.documentsui'],
    'p.android.gm': proc['p.android.gm']
})

### Overenie možných duplikátov.

In [ ]:
attributes.duplicated().sum()

### Zobrazenie základných deskriptívnych údajov vybraných atribútov.

In [25]:
pom = attributes.describe()

#### Funkcia `.describe()` neobsahuje hodnoty: modus a medián, preto ich manuálne pridáme do DF.

### Overenie počtu modusov pre jednotlivé atribúty.

In [ ]:
attributes.mode().count()

#### Nakoľko niektoré atribúty majú viacero modusov, rozhodli sme sa pristúpiť k riešeniu spriemerovania, za účelom získania 1 konkrétnej hodnoty.

### Vypočítanie modusu a jeho pridanie do DF.

In [27]:
temp = attributes.mode().mean()
temp = pd.DataFrame({'mode': temp})
temp = temp.T
pom2 = pd.concat([pom, temp])

### Vypočítanie mediánu a jeho pridanie do DF.

In [28]:
temp = attributes.median()
temp = pd.DataFrame({'median': temp})
temp = temp.T
pom3 = pd.concat([pom2, temp])


### Overenie pridania modusu a mediánu

In [ ]:
pom3

### Grafické zobrazenie jednotlivých distribúcií atribútov.

In [ ]:
kde_plots = attributes.plot(kind='kde', subplots=True, layout=(4, 3), figsize=(15, 10), legend=False)
kde_plots = kde_plots.flatten()

titles =  attributes.columns

i=0
for i in range(0, 10):
    kde_plots[i].set_title(titles[i])
    kde_plots[i].set_xlabel("usage")

    kde_plots[i].tick_params(axis='x', which='both', labelbottom=True)

plt.tight_layout()
plt.subplots_adjust(hspace=0.5, wspace=0.3)
plt.show()

### Podľa jednotlivých grafov na pohľad vyplýva že atribúty patria do normálnej distribúcie.
### Aby naše tvrdenie bolo potvrdené, zobrazíme atribúty cez QQ plot.

### QQ ploty vybraných atribútov.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(13, 15))

axes = axes.flatten()

for idx, col in enumerate(attributes.columns):
    sm.qqplot(attributes[col], fit=True, line="45", ax=axes[idx])
    axes[idx].set_title(col)

for i in range(len(attributes.columns), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### Atribúty podľa QQ plotov na pohľad takisto patria do normálnej distribúcie.
### Problém však vyvolávajú outlier-y, ktoré však neskôr odstránime.
### Vykonali ešte test na normálnu distribúciu, a to `Shapiro-Wilk` test na normalitu.

In [ ]:
for i in attributes.columns:
    pvalue=stats.shapiro(attributes[i]).pvalue
    print(f"{i:<26} = {pvalue:.6f}")

### Na základe `Shapiro-Wilk` testu sa naše pôvodné tvrdenie, usúdené na základe grafov, vyvrátilo, pretože `p < 0.05` v každom prípade.
### Atribúty teda nepatria do normálnej distribúcie.
### Domnievame sa, že výskyt outlier-ov môže hrať v tomto rolu.

#
### Na základe upozornenia z predchádzajúceho testu na normalitu, sme zopakovali 100-krát rovnaký test, vždy s náhodnou vzorkou veľkosti 1000.

In [ ]:
x = 100
y = 1000

print(f"{'P-values pre vzorky:':<31} {y//2} {y:>9} {y*2:>9}\n")

for i in attributes.columns:
    pvalue, pvalue1, pvalue2 = 0, 0, 0
    for j in range(x):
        sampled_data = attributes[i].sample(n=y//2, random_state=j)
        sampled_data1 = attributes[i].sample(n=y, random_state=j)
        sampled_data2 = attributes[i].sample(n=y*2, random_state=j)

        pvalue += stats.shapiro(sampled_data).pvalue
        pvalue1 += stats.shapiro(sampled_data1).pvalue
        pvalue2 += stats.shapiro(sampled_data2).pvalue
    
    pvalue /= x
    pvalue1 /= x
    pvalue2 /= x

    print(f"{i:<26} = {pvalue:.6f}  {pvalue1:.6f}  {pvalue2:.6f}")

### Z vykonaných testov vidíme, že čím zväčšujeme veľkosť vzorky, tým menej atribútov patrí do normálnej distribúcie.

#
## *C) Párová analýza dát*

### Rozhodli sme sa pre porovnanie jednotlivých atribútov prostredníctvom `Heat mapy`.

### Heat mapa pre Connections.

In [ ]:
con2=con[[  'mwra', 
            'c.dogalize', 'c.android.chrome', 'c.katana',
            'c.android.gm', 'c.android.youtube', 'c.android.vending',
            'c.updateassist', 'c.UCMobile.x86', 'c.UCMobile.intl', 'c.raider']].copy()

con_corr2=con2.corr()

fig, ax = plt.subplots(figsize=(13,11))

ax.set_title('Heat map pre Connections', fontsize=16)

sns.heatmap(con_corr2, annot=True)
plt.show()

### Z vykreslenej `Heat mapy` vidíme, že existujú korelácie medzi viacerými atribútmi. Najvýraznejšie korelácie sú medzi:
- **`mwra`** a **`c.dogalize`** – korelačná hodnota: **-0.56**

- **`c.android.gm`** a **`c.katana`** – korelačná hodnota: **-0.43**

- **`mwra`** a **`c.katana`** – korelačná hodnota: **-0.30**

### Heat mapa pre Processes.

In [ ]:
proc2 = proc[[  'mwra', 
                'p.android.packageinstaller',
                'p.android.externalstorage', 'p.system', 'p.android.chrome',
                'p.android.settings', 'p.android.documentsui', 'p.android.gm',
                'p.katana', 'p.google', 'p.android.gms', 'p.inputmethod.latin',
                'p.process.gapps', 'p.olauncher', 'p.browser.provider', 'p.notifier',
                'p.gms.persistent', 'p.android.defcontainer', 'p.android.vending',
                'p.simulator', 'p.dogalize']].copy()

proc_corr1=proc2.corr()

fig, ax = plt.subplots(figsize=(25,25))

ax.set_title('Heat map pre Processes', fontsize=20)

sns.heatmap(proc_corr1, annot=True)
plt.show()

### Z tejto Heat mapy vidíme viacero silnejších korelácií ako napr.
- **`mwra`** a **`c.p.android.documnetsui`** – korelačná hodnota: **-0.55**

- **`p.android.game`** a **`p.adroid.settings`** – korelačná hodnota: **-0.54**

- **`mwra`** a **`p.android.packageinstaller`** – korelačná hodnota: **-0.52**

### Pre porovnanie závislostí atribútov medzi `Connections` a `Processes` sme tieto DF spojili DF spojili na základe rovnosti: `ts`, `imei` a `mwra` atribútov.

In [ ]:
con_proc = pd.merge(con, proc, on=['ts', 'imei', 'mwra'])
con_proc.columns

### `Heat mapa` pre merge DF: `Connections a Processes`.

In [ ]:
con_proc_corr=con_proc.drop(columns=['ts', 'imei']).corr()

fig, ax = plt.subplots(figsize=(30,13))

con_proc_corr_show=con_proc_corr.loc["c.dogalize":"c.raider", "p.android.packageinstaller":"p.dogalize"]

ax.set_title('Heat map pre Connections & Processes', fontsize=24)

sns.heatmap(con_proc_corr_show, annot=True)
plt.show()

### Na základe `Heat mapy` sme zistili značné korelácie medzi atribútmi:

- **`c.dogalize`** a **`p.android.documentsui`** – korelačná hodnota: **0.62**

- **`c.katana`** a **`p.android.settings`** – korelačná hodnota: **-0.57**

- **`c.dogalize`** a **`p.android.packageinstaller`** – korelačná hodnota: **-0.41**

## *D) Párová analýza dát - predikovaná premenná*

### Koreláciu atribútov s predikovanou premennou sme vykonali v predchádzajúcom kroku.

### Vyberieme korelácie predikovannej premennej s ostatnými atribútmi.

In [38]:
mwra_corr = con_proc_corr['mwra'].drop('mwra')

### Vybrané korelácie usporiadame podľa absolútnych hodnôt.

In [39]:
mwra_corr = con_proc_corr['mwra'].drop('mwra')
mwra_corr = pd.DataFrame({"values": mwra_corr, "absolute_values": mwra_corr.abs()})
mwra_corr.sort_values(by=["absolute_values"], inplace=True, ascending=False)
mwra_corr = mwra_corr["values"]

### Usporiadané korelácie zobrazíme na `Histograme`.

In [ ]:
index = np.arange(len(mwra_corr))

plt.subplots(figsize=(15, 7))

plt.bar(index[mwra_corr >= 0], mwra_corr[mwra_corr >= 0], color="green", label="positive")
plt.bar(index[mwra_corr < 0], mwra_corr[mwra_corr < 0], color="red", label="negative")

plt.xticks(index, mwra_corr.index, rotation=90)
plt.yticks(np.arange(-0.6, 0.7, 0.1))
plt.title('Bar Plot kladných a negatívnych korelácií.')
plt.ylabel("Hodnoty korelácie")
plt.legend()
plt.show()

### Zobrazíme distribúciu atribútov pomocou `KDE` grafu, podľa predikovanej premennej.

In [ ]:
columns = mwra_corr.index[0:9]

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 15))

axes = axes.flatten()

for idx, col in enumerate(columns):
    sns.kdeplot(x=con_proc[col], hue=con_proc['mwra'], fill=True, ax=axes[idx])

for i in range(len(attributes.columns), len(axes)):
    fig.delaxes(axes[i])


plt.tight_layout()
plt.show()

### Ak by sme boli schopní vertikálnou čiarou od seba oddeliť oba grafy, tak by bol atribút dokonalým predikátorom predikovanej premennej.

#
## *E) Dokumentujte Vaše prvotné zamyslenie k riešeniu zadania projektu.*

- ### Závislosť atribútov sme zobrazli pomocou `Heat mapy` v časti `1.1 C)` aj pre `Connections`, aj `Processes`. Takisto sme vypísali najsilnejšie korelácie medzi atribútmi.
- ### Závisloť predikovannej premennej sme zobrazili cez `Histogram` v časti `1.1 D)`.
- ### Kombinovanie záznamov bolo potrebné na zistenie korelácií atribútov z rôznych DF.
- ### Najsilnejšia korelácia je práve medzi atribútmi z rôznych DF.

#
# **1.2 Identifikácia problémov, integrácia a čistenie dát**

#
## *A) Nevhodná štruktúra dát, duplicitné záznamy, nejednotné formáty, chýbajúce, vychýlené hodnoty*

### Názvy atribútov jednotlivých DF.

#### Connections

In [ ]:
con.columns

#### Processes

In [ ]:
proc.columns

#### Devices

In [ ]:
dev.columns

#### Profiles

In [ ]:
prof.columns

### Premenovanie stĺpcov. Osobná preferencia.

In [46]:
con = con.rename(columns={  'ts':'DateTime',
                            'imei':'ID',
                            'mwra':'Malware',
                            'c.dogalize':'Dogalize',
                            'c.android.chrome':'Chrome',
                            'c.katana':'Katana',
                            'c.android.gm':'Gm',
                            'c.android.youtube':'Youtube',
                            'c.android.vending':'Vending',
                            'c.updateassist':'Update_assist',
                            'c.UCMobile.x86':'UCMobile_x86',
                            'c.UCMobile.intl':'UCMobile_intl',
                            'c.raider':'Raider'})

proc = proc.rename(columns={'ts':'DateTime',
                            'imei':'ID',
                            'mwra':'Malware',
                            'p.android.packageinstaller':'Package_installer',
                            'p.android.externalstorage':'External_storage',
                            'p.system':'System',
                            'p.android.chrome':'Chrome',
                            'p.android.settings':'Settings',
                            'p.android.documentsui':'Document_sui',
                            'p.android.gm':'Gm',
                            'p.katana':'Katana',
                            'p.google':'Google',
                            'p.android.gms':'Gms',
                            'p.inputmethod.latin':'InputMethod_latin',
                            'p.process.gapps':'Process_gapps',
                            'p.olauncher':'OLauncher',
                            'p.browser.provider':'Browser_provider',
                            'p.notifier':'Notifier',
                            'p.gms.persistent':'Gms_persistent',
                            'p.android.defcontainer':'Def_container',
                            'p.android.vending':'Vending',
                            'p.simulator':'Simulator',
                            'p.dogalize':'Dogalize'})

dev = dev.rename(columns={  'latitude':'Latitude',
                            'longitude':'Longitude',
                            'store_name':'City',
                            'code':'Country',
                            'location':'Location',
                            'imei':'ID'})

prof = prof.rename(columns={'job':'Job',
                            'current_location':'Coordinates',
                            'imei':'ID',
                            'user_id':'UserID',
                            'address':'Adress',
                            'ssn':'SSN',
                            'company':'Company',
                            'birthdate':'Birthdate',
                            'registration':'Registration',
                            'mail':'Email',
                            'username':'Username',
                            'name':'Name'})

### Preusporiadanie stĺpcov. Osobná preferencia.

In [47]:
con = con[['ID', 'Malware', 'DateTime', 'Chrome', 'Youtube', 'Dogalize', 'Katana', 'Raider', 'Gm', 'Vending', 'Update_assist', 'UCMobile_x86', 'UCMobile_intl']]

proc = proc[['ID', 'Malware', 'DateTime', 'Chrome', 'Google', 'Dogalize', 'Katana', 'Gm', 'Vending', 'System', 'Settings', 'Simulator', 
             'Notifier', 'Gms', 'Gms_persistent', 'Document_sui', 'Package_installer', 'External_storage', 'InputMethod_latin', 'Process_gapps', 'OLauncher', 'Browser_provider', 'Def_container']]

dev = dev[['ID', 'Country', 'City', 'Location', 'Latitude', 'Longitude']]

prof = prof[['ID', 'UserID', 'Username', 'Name', 'Email', 'Birthdate', 'Job', 'Adress', 'SSN', 'Company', 'Registration', 'Coordinates']]

#
#### Funkcia pre `prof['Coordinates']` na oddelenie zemepisnej šírky a dĺžky.

In [48]:
def extract_coordinate(coordinate, which):
    if pd.isna(coordinate):
        return np.nan
    both = coordinate.split(', ')
    cleared = re.sub(r"[^\d.-]", "", both[which])
    return float(cleared)

### Rozdelenie `prof['Coordinates']` na atribúty `Latitude` a `Longitude`.

In [49]:
prof['Latitude'] = prof['Coordinates'].apply(extract_coordinate, args=(0,))
prof['Longitude'] = prof['Coordinates'].apply(extract_coordinate, args=(1,))

#### Atribút `Coordinates` už nie je potrebný v `Profiles`.

In [50]:
prof = prof.drop('Coordinates', axis=1)

#
### Zmena stĺpcov na správny/zodpovedajúci dátový formát.

#### Funkcia, konkrétne pre `prof['Registration']`, kvôli rôznym dátumovým formátom.

In [51]:
def convert_mixed_date(date_str):
    try:
        return pd.to_datetime(date_str)
    except:
        try:
            # (YYYY-MM-DD)
            return pd.to_datetime(date_str, format='%Y-%m-%d')
        except ValueError:
            try:
                # (MM/DD/YYYY, HH:MM:SS)
                return pd.to_datetime(date_str, format='%m/%d/%Y, %H:%M:%S')
            except ValueError:
                try:
                    # (07 Oct 2023)
                    return pd.to_datetime(date_str, format='%d %b %Y')
                except ValueError:
                    return pd.NaT

### Zmena najmä dátumov na formát `datetime`, ako aj zmena číselných atribútov na `int`, kde `float` je nepotrebný.

In [52]:
con['DateTime'] = pd.to_datetime(con['DateTime'])
con['Malware'] = con['Malware'].astype(int)

proc['DateTime'] = pd.to_datetime(proc['DateTime'])
proc['Malware'] = proc['Malware'].astype(int)

prof['Birthdate'] =  pd.to_datetime(prof['Birthdate'])
prof['Registration'] = prof['Registration'].apply(convert_mixed_date)

### Duplikátne záznamy v jednotlivých DF.

#### Connections

In [ ]:
con.duplicated().sum()

#### Processes

In [ ]:
proc.duplicated().sum()

#### Devices

In [ ]:
dev.duplicated().sum()

#### Profiles

In [ ]:
prof.duplicated().sum()

### Vymazanie duplikátnych záznamov.
#### Duplikáty v Connections a Processes sme odstránili v časti `1.1 B)`. `Profiles` duplikáty neobsahujú.

In [57]:
# con = con.loc[~con.duplicated()].reset_index(drop=True).copy()
# proc = proc.loc[~proc.duplicated()].reset_index(drop=True).copy()
dev = dev.loc[~dev.duplicated()].reset_index(drop=True).copy()

#### Pomocná funkcie na porovnanie vymazných záznamov.

In [58]:
def show_comparison(before, after):

    names = ['Connections', 'Processes', 'Devices', 'Profiles']

    print(f"{'Tabuľka':<12} {'Pred':>8} {'Po':>8} {'Vymazaných':>10}")
    print("="*42)

    for i in range(len(names)):
        pred = before[i]
        po = after[i]
        rozdiel = pred - po
        print(f"{names[i]:<12} {pred:>8} {po:>8} {rozdiel:>10}")

#### Uloženie počtu riadkov po vymazaní duplikátov.

In [59]:
after = count_of_rows()

### Porovnanie po vymazaní duplikátov.

In [ ]:
show_comparison(before, after)

#
## *B) Chýbajúce hodnoty*

### Chýbajúcich hodnoty v jednotlivých DF.

#### Connections

In [ ]:
con.isna().sum()

#### Processes

In [ ]:
proc.isna().sum()

#### Devices

In [ ]:
dev.isna().sum()

#### Profiles

In [ ]:
prof.isna().sum()

#
### Lokalizovanie chýbajúcej hodnoty v `dev['Country']`.

In [ ]:
dev.loc[dev['Country'].isna()]

### Nahradenie chýbajúcej hodnoty v `dev['Country']`. Mesto `Okahandja` sa nachádza v Namíbií = `NA`.

In [66]:
dev.loc[2525, 'Country'] = 'NA'

#### Overenie zmeny.

In [ ]:
dev.loc[dev['Country'] == 'NA']

#
### Atribút `prof['Job']` obsahuje `1762` chýbajúcich hodnôt čo je viac ako polovica z celkového počtu (`2517`) v `Profiles`.
#### Nahradiť chýbajúce hodnoty napr. cez modus alebo zadaním defaultnej hodnoty by pri ďalšej analýze neposkytlo nič relevantné podľa nás.
#### Takisto nevidíme dôvod ponechať tento atribút pre ďalšiu analýzu. Preto sme sa rozhodli atribút odstrániť.

### Odstránenie atribútu `Job` z Profiles.

In [68]:
prof = prof.drop('Job', axis=1)

#
### Atribút `prof['Birtdate']` obsahuje `1133` chýbajúcich hodnôt, takmer polovica (`2517`).
#### Z rovnakých dôvodov ako pri atribúte `Birthdate` sme sa rozhodli atribút odstrániť.

### Odstránenie atribútu `Birthdate` z Profiles.

In [69]:
prof = prof.drop('Birthdate', axis=1)

#
### Atribút `prof['Adress']` obsahuje `378` chýbajúcich hodnôt, čo je približne `15%`.
### Chýbajúce hodnoty pre tento atribút sme sa rozhodli nahradiť defaultnou hodnotou `'not included'`.

### Zmena chýbajúcich hodnôt na `'not included'`.

In [70]:
prof['Adress'] = prof['Adress'].fillna('not included')

#
### Atribúty `prof['Latitude']` a `prof['Longitude']` obsahujú `126` chýbajúcich hodnôt (dvojíc), čo je približne `5%`.
### Po detailnejšom skúmaní a zisťovaní skutočnej polohy koordinácií, sme zistili, že veľké percento ukazuje na náhodné miesta v oceánoch po celom svete a na miesta, ktoré jednoducho nedávajú zmysel pre človeka.
### Z toho dôvodu sme sa rozhodli zbaviť týchto atribútov.

### Zobrazenie súradníc je vykreslené na grafe spolu s **relatívnym** vyzobrazením kontinentov (Európa a Ázia sú osobitne).

In [ ]:
plt.figure(figsize=(13, 6))
plt.scatter(prof['Longitude'], prof['Latitude'], color='black', s=10, alpha=0.5)

asia = plt.Rectangle((36, 31), 105, 40, linewidth=2, edgecolor='orange', facecolor='none', label='Ázia')
africa = plt.Rectangle((-15, -35), 50, 69, linewidth=2, edgecolor='brown', facecolor='none', label='Afrika')
europe = plt.Rectangle((9, 36.5), 21, 23, linewidth=2, edgecolor='blue', facecolor='none', label='Európa')
south_america = plt.Rectangle((-78, -55.5), 40.9, 65, linewidth=2, edgecolor='green', facecolor='none', label='Južná Amerika')
north_america = plt.Rectangle((-138, 32.7), 72.4, 36.5, linewidth=2, edgecolor='red', facecolor='none', label='Severná Amerika')

plt.gca().add_patch(africa)
plt.gca().add_patch(asia)
plt.gca().add_patch(europe)
plt.gca().add_patch(south_america)
plt.gca().add_patch(north_america)

plt.xticks(np.arange(-180, 181, 30))
plt.yticks(np.arange(-90, 91, 15))

plt.xlabel('Zemepisná dĺžka')
plt.ylabel('Zemepisná šírka')
plt.title('Geografické oblasti')

plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

### Z grafu je očividné, že takmer každý kút sveta je zastúpený.

#
### Odstránenie atribútútov `Latitude` a `Longitude` z Profiles.

In [72]:
prof = prof.drop(['Latitude', 'Longitude'], axis=1)

#
## *C) Vychýlené hodnoty*

#### Funkcia na vykreslenie box-plotov pre ukážku outlier-ov.

In [73]:
def show_box_plots(df, cols, columns):
    rows = (len(columns) + cols - 1) // cols

    plt.figure(figsize=(5 * cols, 5 * rows))

    for i, col in enumerate(columns):
        plt.subplot(rows, cols, i + 1)
        try:
            df[col].plot(kind='box')
            plt.title(f'{col}')
        except Exception as e:
            print(f"Chyba pri vykresľovaní boxplotu pre {col}: {e}")
        plt.ylabel('Hodnoty')

    plt.tight_layout()
    plt.show()

### Box-ploty pre `Connections`.

In [ ]:
show_box_plots(con, 5, con.columns[3:])

#### Z grafov môžeme vidieť, že atribúty `Raider`, `UCMobile_x86`, `UCMobile_intl` nemajú žiadne viditeľné outlier-y.
#### Na druhej strane môžeme vidieť zaujímavé hodnoty pri atribúte `Update_assist`. Tomuto atribútu sa viac budeme venovať neskôr.

#
### Box-ploty pre `Processes`.

In [ ]:
show_box_plots(proc, 5, proc.columns[3:])

#### Vyzobrazením grafov pre `Processes` vidíme že až takmer polovica atribútov (9/20) nemá viditeľné outlier-y.
#### Na všetky atribúty a ich konkrétny počet ako aj percentuálne vyjadrenie outlier-ov sa pozrieme nižšie.

#
#### Funkcia identifikovania outlier-ov.

In [76]:
def identify_outliers(column, col_name):
    lower = column.quantile(0.25) - 1.5 * stats.iqr(column)
    upper = column.quantile(0.75) + 1.5 * stats.iqr(column)

    outliers = column[(column > upper) | (column < lower)]
    
    if not outliers.empty:
        return (col_name, outliers.count())
    else:
        return (col_name, 0)

#### Funkcia na vypísanie a vrátenie outlier-ov.

In [77]:
def count_outliers_and_print(df, dataframe_name):
    column_names = df.columns[3:]
    df_count = df.shape[0]

    values = []
    
    for col in column_names:
        values.append(identify_outliers(df[col], col))
    
    sorted_outliers = sorted(values, key=lambda x: x[1], reverse=True)
    
    print(f"Počet outlierov (% aj počet) pre {dataframe_name} ({df_count}):\n")
    for col, count in sorted_outliers:
        percentage = round((count / df_count) * 100, 1)
        print(f'{col:>18}  {percentage:<4}  {count:<6}')
    
    return values

#
### Outlier-y pre `Connections`.

In [ ]:
con_outliers = count_outliers_and_print(con, "Connections")

#### Z vypočítaných hodnôt vidíme, že atribúty DF `Connections`, až na jeden atribút, neobsahujú veľa outlier-ov.
#### Problémový atribút je `Update_assist` s počtom outlier-ov až `15.9%` - `2379` záznamov.
#### Rozhodli sme sa, že okrem atribútu `Update_assist`, všetkým atribútom outlier-y zmeníme na hraničné hodnoty.
#### Na atribút `Update_assist` sa pozrieme ešte bližšie neskôr.

#
### Outlier-y pre `Processes`.

In [ ]:
proc_outliers = count_outliers_and_print(proc, "Processes")

#### Zo zistených hodnôt vidíme, že `Processes` nemá s outlier-mi veľký problém.
#### Najviac outlier-ov má atribút `External_storage` - `1.7%` - `249` záznamov.
#### Rozhodli sme sa, že každému atribútu, s `počtom outlier-ov > 0`, hodnoty outlier-ov zmeníme na hraničnú hodnotu.  

#
#### Funkcia na zmenu outlier-ov na hraničné hodnoty.

In [80]:
def correct_outliers_by_column(df, df_package, exceptions):
    for data in df_package:
        col_name = data[0]
        count = data[1]

        if col_name in exceptions: continue
    
        if count > 0:
            lower = df[col_name].quantile(0.25) - 1.5 * stats.iqr(df[col_name])
            upper = df[col_name].quantile(0.75) + 1.5 * stats.iqr(df[col_name])

            df[col_name] = df[col_name].apply(
                lambda x: lower if x < lower else upper if x > upper else x
            )
    
    return df

#
### Zmena hodnôt outlier-ov na hraničné hodnoty pre `Connections`, okrem atribútu `Update_assist`.

In [81]:
con = correct_outliers_by_column(con, con_outliers, ["Update_assist"])

#
### Zmena hodnôt outlier-ov na hraničné hodnoty pre `Processes`.

In [82]:
proc = correct_outliers_by_column(proc, proc_outliers, [])

#
### Bližší pohľad na `Connections['Update_assist']`.

In [ ]:
y_limits = [
    (0, 100, 10),
    (0, 15, 1),
    (0, 5, 0.5),
    (0, 2, 0.2),
    (0, 0.5, 0.05),
    (0, 0.1, 0.01)
]

upper_border = con['Update_assist'].quantile(0.75) + 1.5 * stats.iqr(con['Update_assist'])

plt.figure(figsize=(15, 7))

for i, (y_min, y_max, step) in enumerate(y_limits):
    plt.subplot(1, 6, i + 1)
    
    con['Update_assist'].plot(kind='box')
    
    plt.ylim(y_min, y_max)
    plt.yticks(np.arange(y_min, y_max + step, step))

    outliers_count = con[(con['Update_assist'] > upper_border) & (con['Update_assist'] <= y_max)].shape[0]
    
    plt.title(f'Outliers: {outliers_count} - {round((outliers_count/con['Update_assist'].shape[0]) * 100, 1)}%')

plt.tight_layout()
plt.show()

### Môžeme vidieť ktoré hodnoty a aké rozmedzie outlier-y obsadzujú.
### Po dlhšom rozhodovaní sme sa rozhodli, že odstánime outlier-y s hodnotou nad `5`.
### Domnievame sa, že veľké množstvo outlier-ov pri tomto atribúte môže predstavovať niečo viac.
### Konkrétne môže ísť o extrémne hodnoty, ktoré boli zaznamenané, ale nie sú chybné vzhľadom na počet.
### Ako sme z grafov mohli vidieť tak väčšina hodnôt `con['Update_assist']` je koncentrovaná okolo hodnoty `0.01`.
### V porovnaní, hodnota, kde začínajú outliery je `0.03` a tiahne sa až k hodnote `1.2` (4. graf zľava)
### Odstrániť všetky outlier-y sme neuznali za vhodné ako aj nahradiť celú respektíve určitú časť na hraničnú hodnotu.
### Veríme, že ponechať hodnoty s menšou úpravou, bude najlepšie.

#
### Odtránenie záznamov, kde `con['Update_assist'] >= 5`.

In [84]:
con = con[con['Update_assist'] < 5].copy()

#
# **1.3 Formulácia a štatistické overenie hypotéz o dátach**

#
## *A) Sformulujte dve hypotézy o dátach v kontexte zadanej predikčnej úlohy.*

### `Hypotéza 1`:

#### `H0`: `con['Youtube']` má v priemere nižšiu váhu ako `con['Dogalize']` v malware-related-activity stave.
#### `HA`: `con['Youtube']` má váhu vyššiu alebo `con['Dogalize']` má váhu nižšiu v malware-related-activity stave voči oponentovi.

#
### Počet záznamov v `Connections`, kde `mwra=1`.

In [ ]:
con.query('Malware == 1').shape[0]

### Zistíme priemernú hodnotu pre `con['Youtube']` ak `mwra=1`.

In [ ]:
ytb_mean = con[con['Malware'] == 1]['Youtube'].mean(); print(ytb_mean)

### Zistíme priemernú hodnotu pre `con['Dogalize']` ak `mwra=1`.

In [ ]:
dogalize_mean = con[con['Malware'] == 1]['Dogalize'].mean(); print(dogalize_mean)

### Z vypočítaných hodnôt vidíme, že priemerná hodnota `Youtube` > `Dogalize`

In [ ]:
print(abs(ytb_mean - dogalize_mean))

### Z hodnôt vyššie `H0` zamietame a prijímame `HA`.

#
### `Hypotéza 2`:

#### `H0`: Najsilnejšie korelácie atribútov z `Conenctions` a `Processes` s predikovanou premennou sa nezmenili o viac ako `2%`. 
#### `HA`: Najsilnejšie korelácie atribútov z `Conenctions` a `Processes` s predikovanou premennou sa zvýšili alebo znížili o viac ako `2%`.

#
### Vypočítame vzájomné korelácie atribútov z `Conenctions` a `Processes`.

In [89]:
con_proc_new = pd.merge(con, proc, on=['DateTime', 'ID', 'Malware'], suffixes=('_Con', '_Proc'))

con_proc_corr_new=con_proc_new.drop(columns=['DateTime', 'ID']).corr()

### Vyberieme len korelácie s predikovanou premennou a graficky ich zobrazíme zoradené podľa absolútnej hodnoty.

In [ ]:
mwra_corr_new = con_proc_corr_new['Malware'].drop('Malware')


mwra_corr_new = pd.DataFrame({"values": mwra_corr_new, "absolute_values": mwra_corr_new.abs()})
mwra_corr_new.sort_values(by=["absolute_values"], inplace=True, ascending=False)
mwra_corr_new = mwra_corr_new["values"]

index = np.arange(len(mwra_corr_new))

plt.subplots(figsize=(15, 7))

plt.bar(index[mwra_corr_new >= 0], mwra_corr_new[mwra_corr_new >= 0], color="green", label="positive")
plt.bar(index[mwra_corr_new < 0], mwra_corr_new[mwra_corr_new < 0], color="red", label="negative")

plt.xticks(index, mwra_corr_new.index, rotation=90)
plt.yticks(np.arange(-0.6, 0.7, 0.1))
plt.title('Bar Plot kladných a negatívnych korelácií.')
plt.ylabel("Hodnoty korelácie")
plt.legend()
plt.show()

### Vidíme, že poradie 9 najsilnejších korelácií s predikovanou premennou zostalo rovnaké.
### Na základe toho, môžeme priamo porovnať tieto korelácie s pôvodnými koreláciami.

#
### Pre lepšie porovnanie rozdielov korelácií sme vypočítané hodnoty zobrazili číselne.

In [ ]:
print(f'{"attribute":>14} {"before":>23} {"now":>10} {"diff":>12} {"% diff":>10}\n')
i=0
for i in range(len(mwra_corr.index)):
    name = mwra_corr.index[i]

    difference = abs(mwra_corr_new.iloc[i]-mwra_corr.iloc[i])
    percent_diff = abs(difference / mwra_corr.iloc[i] * 100)

    prefix = "-" if mwra_corr_new.iloc[i] < 0 else " "

    print(f'{name:<27} |  {prefix}{abs(mwra_corr.iloc[i]):.5f}  |  {abs(mwra_corr_new.iloc[i]):.5f}  |  {difference:.5f}  |  {percent_diff:.1f}')

    i+=1
    if i==9:
        break

### Z hodnôt vidíme, že percentuálny rozdiel v koreláciách je menší ako `1%`.
### Na základe toho `H0` neodmietame.

#
## *B) Overte či Vaše štatistické testy majú dostatočne silnú štatistickú silu.*

### Veľkosti vzoriek:
- #### Pri `Hypotéze 1`, kde je vzorka DF `Connections` sme počas EDA odstránili celkovo 22 záznamov. Takisto sme v `Youtube` 37 outlier-ov zmenili na hraničné hodnoty a v `Dogalize` 35 outlier-ov. Pri celkovom počte záznamov `14 961` to predstavuje nízky vplyv.
###
- #### Pri `Hypotéze 2`, kde sme analyzovali atribúty z DF `Connections` (14961 záz.) a `Processes` (14983 záz.), sa veľkosť vzorky zmenila rovnako ako v `Hyp. 1` len pre `Connections`. Takisto sme v `Processes` v 9 atribútoch menili hodnoty outlier-ov na hraničné hodnoty. Počet outlierov však nepresiahol `2%` v každom atribúte.
###
### Nevidíme dôvod, aby naše testy nemali nedostatočnú štatistickú silu.

#
# EXTRA

In [92]:
# con.to_csv('dataset82_edited/Connections.csv', index=False)
# proc.to_csv('dataset82_edited/Processes.csv', index=False)
# dev.to_csv('dataset82_edited/Devices.csv', index=False)
# prof.to_csv('dataset82_edited/Profiles.csv', index=False)